# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Introducción al aprendizaje automático

### Trabajo práctico entregable - Grupo 22 - Parte 1: Regresión en California

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Dataset

### Lectura del dataset

Cargamos el conjunto de datos y vemos su contenido.

In [ ]:
from sklearn.datasets import fetch_california_housing

# Treamos la data (matriz con los datos de entrada / atributos) en X
# y el target (vector de valores a predecir) en y, ambos como pandas DataFrame
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

# Traemos todo el dataset, con toda su descripción
california = fetch_california_housing()

In [ ]:
print(california['DESCR'])  # Descripción del dataset

In [ ]:
display(X.head())
X.shape

In [ ]:
display(y.head())
y.shape

### Segmentación del dataset: conjunto de entrenamiento y conjunto de evaluación

Dividimos aleatoriamente los datos en 80% para entrenamiento y 20% para evaluación. Nos quedan entonces 16.512 registros para entrenar y 4.128 para evaluar.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
X_train.shape, X_test.shape

---
# Ejercicio 1 - Descripción cualitativa

### Pregunta 1 - ¿De qué se trata el conjunto de datos?

Utilizaremos el conjunto de datos *California house prices dataset* disponible en `sklearn`, obtenido a su vez del repositorio `StatLib`. En el mismo se ha recolectado información basada en el censo de California de 1990, donde cada fila representa un **grupo de bloques** (aunque quizás no sea totalmente correcto, de ahora en más nos referiremos a esto como **distrito**), *i.e.* la menor unidad geográfica para la cual la oficina de censos de EE.UU. publica datos, conteniendo entre 600 y 3.000 personas. Por otro lado, una vivienda (*household*) implica un grupo de personas que viven dentro de la misma casa.

### Pregunta 2 - ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?

La variable que pretende predecir este dataset nos indica la mediana del valor de las casas para los diferentes distritos en California, expresados en US$ 100.000.

### Pregunta 3 - ¿Qué información (atributos) hay disponibles para hacer la predicción?

En el dataset tenemos 8 variables predictoras:
1. MedInc: mediana del salario dentro del distrito.
2. HouseAge: mediana de la antigüedad de las casas dentro del distrito.
3. AveRooms: cantidad promedio de habitaciones por vivienda.
4. AveBedrms: cantidad promedio de cuartos por vivienda.
5. Population: población dentro del distrito.
6. AveOccup: cantidad promedio de personas que viven dentro de la vivienda.
7. Latitude y Longitude: ubicación geográfica del distrito.

Para cada uno de estos atributos, contamos con 20.640 registros, lo cual nos lleva a una matriz de atributos de tamaño 20.640 $\times$ 8.

### Pregunta 4 - ¿Qué atributos imagina ud. que serán los más determinantes para la predicción?

Dado que el conjunto de datos pretende predecir el costo de las casas según el distrito, imaginamos que los atributos más influyentes serán:
- MedInc: a mayor ingreso, mayor es el valor de la vivienda a la que alguien puede aspirar. 
- HouseAge: cuanto más vieja la propiedad, esta puede ser más barata.
- Latitude y Longitude: seguramente existan diferencias en los precios entre barrios y eso nos lo puede dar la latitud y longitud.

### Pregunta 5 - ¿Qué problemas observa a priori en el conjunto de datos? ¿Observa posibles sesgos, riesgos, dilemas éticos, etc? Piense que los datos pueden ser utilizados para hacer predicciones futuras.

Dado que se tienen los valores promedios de habitaciones y cuartos por vivienda, los autores previenen que las columnas pueden presentar valores extremos por distrito con pocas viviendas así como muchas viviendas vacías, *e.g.* lugares para vacacionar.

> **¿Algún otro problema?**

---
# Ejercicio 2 - Visualización

A continuación se realiza un scatterplot para la variable objetivo en función de cada uno de los atributos.

A simple vista, se puede sacar las siguientes conclusiones:
* Hay una relación creciente entre la variable MedInc (promedio del ingreso) y la variable objetivo.
* En el caso de la Latitud y Longitud se puede observar que existen algunos rangos para los cuales la variable objetivo toma valores más bajos que para otros. 
Esto se puede observar para latitudes cercanas a 36 y superiores a 39 y para longitudes cercanas a -120, menores a -123 y mayores a -116.

Para el resto de las variables es difícil sacar una conclusión con este tipo de gráficos ya que son muy ruidosos y, particularmente, AveRooms, AveBedrms, Population y AveOccup presentan valores extremos que terminan ocultando posibles tendencias.

In [ ]:
features = X.columns

row, col = 0, 0
fig, axs = plt.subplots(2,4, figsize=(16, 10))  
for i in range(len(features)):
    row = i // 4  # Calculate the row index
    col = i % 4   # Calculate the column index
    axs[row, col].scatter(X[features[i]], y, facecolor="dodgerblue", edgecolor="k", label="datos")
    axs[row, col].grid()
    axs[row, col].set_axisbelow(True)  # Set the grid behind the plot
    # Add y-axis label only for the first column in each row
    if col == 0:
        axs[row, col].set_ylabel("MedHouseVal")
    axs[row, col].set_xlabel(features[i])  # Add subplot title
plt.show()

A continuación se realizó otra representación. Cada atributo se dividió en 8 cuantiles y se realizaron boxplots de la variable objetivo para observar sus diferencias.

La ventaja de estos gráficos es que podemos observar las diferencias en las medianas para cada cuantil así como la dispersión de valores.

A ojo se puede concluir:
* MedInc: La mediana para cada cuantil es creciente, lo cual refuerza lo visto en el scatterplot y la conclusión de que esta variable y la variable objetivo poseen una relación creciente.
* HouseAge: No parece haber una dependencia importante en la variable objetivo con la antigüedad promedio de las casas. Se puede observar un ligero crecimiento en la mediana para el último cuantil.
* AveRooms: Para los cuantiles 0 a 4 no se aprecia una diferencia de comportamiento para la variable objetivo. Sin embargo, a partir del cuantil 5 se observa un crecimiento en la mediana lo que indica una relación creciente entre ambas variables.
* AveBedrms: No parece haber una dependencia importante en la variable objetivo con la edad promedio de las casas. Se puede observar un ligero decrecimiento en la mediana para los últimos 3 cuantiles.
* Population: No parece haber una dependencia importante en la variable objetivo con la población. Se puede observar una disminución en la dispersión para los últimos cuantiles lo que indica que para se alcanzan precios menores que para el resto.
* AveOccup: Se observa una ligera disminución en la mediana para cada cuantil, además de una disminución en la dispersión. Esto permite intuir que debe haber una dependencia decreciente entre la variable objetivo y la ocupación promedio.
* Latitude: Se puede observar que para los cuantiles 4 y 7 la variable objetivo tiende a ser menor que para el resto de los cuantiles.
* Longitude: Se puede observar que para los cuantiles 2, 3 y 7 la variable objetivo tiende a ser menor que para el resto de los cuantiles.

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(16, 10))  
for i in range(len(features)):
    row = i // 4  # Calculate the row index
    col = i % 4   # Calculate the column index
    X['quantile'] = pd.qcut(X[features[i]], 8)
    sns.boxplot(X, x=X['quantile'], y=y, ax=axs[row,col])
    axs[row, col].grid()
    axs[row, col].set_axisbelow(True)  # Set the grid behind the plot
    # Add y-axis label only for the first column in each row
    if col == 0:
        axs[row, col].set_ylabel("MedHouseVal")
    axs[row, col].set_xlabel(features[i])  # Add subplot title
    axs[row, col].set_xticklabels(axs[row, col].get_xticks())
plt.show()

A partir de las conclusiones anteriores, creemos que el orden de importancia (de mayor a menor) es el siguiente:
1. MedInc
2. AveRooms
3. Longitude y Latitude (tienen igual peso)
4. AveOccup
5. HouseAge
6. Population
7. AveBedrms

---
# Ejercicio 3 - Regresión Lineal

> Consigna: inicio.

1. Seleccione **un solo atributo** que considere puede ser el más apropiado.
2. Instancie una regresión lineal de **scikit-learn**, y entrénela usando sólo el atributo seleccionado.
3. Evalúe, calculando error cuadrático medio para los conjuntos de entrenamiento y evaluación.
4. Grafique el modelo resultante, junto con los puntos de entrenamiento y evaluación.
5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

**Observación:** Con algunos atributos se puede obtener un error en test menor a 50.

> Consigna: fin.

Se selecciona la columna MedInc, considerada la más importante para la predicción

In [ ]:
feature = 'MedInc'  # selecciono el atributo 'MedInc'
X_train_f = X_train[feature]
X_train_f = X_train_f.values.reshape(-1, 1)
X_test_f = X_test[feature]
X_test_f = X_test_f.values.reshape(-1, 1)
X_train_f.shape, X_test_f.shape

Se instancia una regresión lineal de scikit-learn y se entrena con el atributo seleccionado

In [ ]:
lr = LinearRegression()  # el bias ya esta como feature
lr.fit(X_train_f, y_train)

Se evalúa el modelo utilizando el set de testeo

In [ ]:
y_pred_train = lr.predict(X_train_f)
train_error = mean_squared_error(y_train, y_pred_train)

y_pred_test = lr.predict(X_test_f)
test_error = mean_squared_error(y_test, y_pred_test)
print(f'Error de entrenamiento: {train_error}')
print(f'Error de testeo: {test_error}')

Se grafica el modelo resultante, junto con los puntos de entrenamiento y evaluación

In [ ]:
x_start = min(np.min(X_train_f), np.min(X_test_f))
x_end = max(np.max(X_train_f), np.max(X_test_f))
x = np.linspace(x_start, x_end, 200).reshape(-1, 1)

plt.plot(x, lr.predict(x), color="tomato", label="modelo")
plt.scatter(X_train_f, y_train, facecolor="dodgerblue", edgecolor="k", label="Entrenamiento")
plt.scatter(X_test_f, y_test, facecolor="white", edgecolor="k", label="Evaluación")
plt.title(feature)
plt.legend()
plt.show()

---
# Ejercicio 4 - Regresión Polinomial

> Consigna: inicio.

En este ejercicio deben entrenar regresiones polinomiales de diferente complejidad, siempre usando **scikit-learn**.

Deben usar **el mismo atributo** seleccionado para el ejercicio anterior.

1. Para varios grados de polinomio, haga lo siguiente:
    1. Instancie y entrene una regresión polinomial.
    2. Prediga y calcule error en entrenamiento y evaluación. Imprima los valores.
    3. Guarde los errores en una lista.
2. Grafique las curvas de error en términos del grado del polinomio.
3. Interprete la curva, identificando el punto en que comienza a haber sobreajuste, si lo hay.
4. Seleccione el modelo que mejor funcione, y grafique el modelo conjuntamente con los puntos.
5. Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

**Observación:** Con algunos atributos se pueden obtener errores en test menores a 40 e incluso a 35.

> Consigna: fin.

En este ejercicio vamos a seguir usando el atributo MedInc como en el ejercicio anterior, pero estudiaremos qué ocurre a medida que aumentamos el grado del polinomio regresor. Hacemos entonces un barrido del grado del polinomio, obteniendo el error cuadrático medio (MSE) para cada caso, tanto en el set de entrenamiento como en el de evaluación.

In [ ]:
Err_train = []
Err_test = []
degrees = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 50, 75, 100]

for degree in degrees:
    # Entrenamiento
    lr = LinearRegression()
    poly = PolynomialFeatures(degree)
    Z_train = poly.fit_transform(X_train_f, y_train)
    lr.fit(Z_train, y_train)
    
    # Predicción
    y_pred_train = lr.predict(Z_train)
    Z_test = poly.fit_transform(X_test_f, y_test)
    y_pred_test = lr.predict(Z_test)

    # Métricas
    Err_train.append(mean_squared_error(y_train, y_pred_train))
    Err_test.append(mean_squared_error(y_test, y_pred_test))

Graficamos ahora la variación del MSE en función del grado del polinomio, para ambos conjuntos de datos. Vemos que rápidamente el valor de MSE cae al comienzo. Luego, la disminución continua, pero de manera más gradual hasta el grado 15. Hasta acá las diferencias entre el conjunto de entrenamiento y el de evaluación son pequeñas, siendo mayores para el caso de evaluación, como era de esperarse. A partir de ahí pega un primer salto, donde las curvas de entrenamiento y evalación crecen en paralelo hasta el grado 40, donde el MSE del conjunto de entrenamiento alcanza un *plateau*, mientras que el del conjunto de evaluación sigue creciendo paulatinamente hasta que se dispara alrededor del grado 75. Observamos entonces que se presenta sobreajuste a partir del grado 40.

Ahora, para elegir el grado óptimo del polinomimo, nos centraremos en la región con grado menor a 16, donde los errores son menores. Si bien el mínimo estricto de la curva de entrenamiento se alcanza con grado 10, observamos que la diferencia no es significativa respecto al error del polinomio de grado 7. Recurriendo a la navaja de Ockham, consideramos entonces que el mejor modelo es con grado 7.

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(18, 5))

axs[0].plot(degrees, Err_train, color="blue", label="Entrenamiento")
axs[0].plot(degrees, Err_test, color="red", label="Evaluación")
axs[0].legend()
axs[0].set_xlabel("Grado del polinomio")
axs[0].set_ylabel("MSE")
axs[0].set_xlim(-2, 102)
axs[0].set_ylim(0, 12)

axs[1].plot(degrees, Err_train, color="blue", label="Entrenamiento")
axs[1].plot(degrees, Err_test, color="red", label="Evaluación")
axs[1].legend()
axs[1].set_xlabel("Grado del polinomio")
axs[1].set_ylabel("MSE")
axs[1].set_xlim(30, 80)
axs[1].set_ylim(1.1, 2)

axs[2].plot(degrees, Err_train, color="blue", label="Entrenamiento")
axs[2].plot(degrees, Err_test, color="red", label="Evaluación")
axs[2].legend()
axs[2].set_xlabel("Grado del polinomio")
axs[2].set_ylabel("MSE")
axs[2].set_xlim(0, 16)
axs[2].set_ylim(0.66, 0.72)

plt.show()

Volvemos a entrenar el modelo, pero fijando el grado en 7, para así graficar los resultados.


**FALTA**
Interprete el resultado, haciendo algún comentario sobre las cualidades del modelo obtenido.

In [ ]:
# Mejor modelo
degree = 7
lr = LinearRegression()
poly = PolynomialFeatures(degree)
Z_train = poly.fit_transform(X_train_f, y_train)
lr.fit(Z_train, y_train)

In [ ]:
plt.plot(x, lr.predict(poly.fit_transform(x)), color="tomato", label="modelo")
plt.scatter(X_train_f, y_train, facecolor="dodgerblue", edgecolor="k", label="Entrenamiento")
plt.scatter(X_test_f, y_test, facecolor="white", edgecolor="k", label="Evaluación")
plt.title(feature)
plt.legend()
plt.show()

---
# Ejercicio 5 - Regresión con más de un Atributo

> Consigna: inicio.

En este ejercicio deben entrenar regresiones que toman más de un atributo de entrada.

1. Seleccione **dos o tres atributos** entre los más relevantes encontrados en el ejercicio 2.
2. Repita el ejercicio anterior, pero usando los atributos seleccionados. No hace falta graficar el modelo final.
3. Interprete el resultado y compare con los ejercicios anteriores. ¿Se obtuvieron mejores modelos? ¿Porqué?

> Consigna: fin.

In [ ]:
# 1. Resolver acá. Ayuda (con dos atributos):
selector = (np.array(california['feature_names']) == 'HouseAge') | (np.array(california['feature_names']) == 'AveRooms')

X_train_fs = X_train[:, selector]
X_test_fs = X_test[:, selector]
X_train_fs.shape, X_test_fs.shape

In [ ]:
# 2. Resolver acá.